<a href="https://colab.research.google.com/github/fllay/lab/blob/main/lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi 

In [ ]:
!pip install flask
!pip install flask_cors

In [ ]:
# baseline cnn model for mnist
from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.model_selection import KFold
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model

In [ ]:
# imports for array-handling and plotting
import numpy as np
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt

# let's keep our keras backend tensorflow quiet
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
# for testing on CPU
#os.environ['CUDA_VISIBLE_DEVICES'] = ''

# keras imports for the dataset and building our neural network
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils

load dataset

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

show some data

In [ ]:
fig = plt.figure()
for i in range(9):
  plt.subplot(3,3,i+1)
  plt.tight_layout()
  plt.imshow(X_train[i], cmap='gray', interpolation='none')
  plt.title("Digit: {}".format(y_train[i]))
  plt.xticks([])
  plt.yticks([])
fig

In [ ]:
#reshape data to fit model
X_train = X_train.reshape(60000,28,28,1)
X_test = X_test.reshape(10000,28,28,1)

In [ ]:
from keras.utils import to_categorical
#one-hot encode target column
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_train[0]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
#create model
model = Sequential()



#add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(28,28,1)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))



In [ ]:
#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
#train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

In [ ]:
#predict first 4 images in the test set
np.argmax(model.predict(X_test[:4]), axis=-1)

In [ ]:
y_test[:4]

Save model

In [ ]:
model.save('final_model.h5')

In [ ]:
!wget https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2019/02/sample_image.png

In [ ]:
def load_image(filename):
	# load the image
	img = load_img(filename, color_mode = "grayscale", target_size=(28, 28))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 1 channel
	img = img.reshape(1, 28, 28, 1)
	# prepare pixel data
	img = img.astype('float32')
	img = img / 255.0
	return img

In [ ]:
# load the image
img = load_image('sample_image.png')
# load model
model = load_model('final_model.h5')
# predict the class
#digit = model.predict_classes(img)
digit = np.argmax(model.predict(img), axis=-1)
#digit = np.argmax(model.predict(img), axis=-1)
print(digit[0])

In [ ]:
!git clone https://github.com/fllay/lab.git

In [ ]:
from flask import Flask, render_template, request, abort, jsonify, send_file
from flask_cors import CORS
import os
TEMPLATE_DIR = os.path.abspath('/content/lab/lab1')

app = Flask(__name__, template_folder=TEMPLATE_DIR)
#CORS(app)

@app.route('/')
def index():
    return render_template('index.html')
    #return 'Web App with Python Flask!'

@app.route('/hello', methods=['GET'])
def hello():
    print("hello called")
    message = {
          'message': "Hello world222"
    }
    return jsonify(message)

from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))
app.run(host='0.0.0.0', port=5000)




In [ ]:
!pip install wget

# Exercise



*   Create a route for training a model with number of epocs and batch size and save the model to a file. To test the API, create a button in the web page to call the API. 
*   Given an image link, create a route to predict the digit and show result on the web interface. The code below may be useful



In [ ]:
import requests
import shutil

r = requests.get('https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2019/02/sample_image.png',
                 stream=True, headers={'User-agent': 'Mozilla/5.0'})

#r = requests.get('https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2019/02/sample_image.png', stream=True)
print(r.status_code)
if r.status_code == 200:
    print("OK")
    with open("/content/img.png", 'wb') as f:
        r.raw.decode_content = True
        shutil.copyfileobj(r.raw, f)